In [11]:
import os
import pickle
import json
import csv
import time
import lyricsgenius as lg
import re
import numpy as np
import pandas as pd

#import credential
import requests

In [14]:
CLIENT_ACCESS_TOKEN = "xXeju4S5sFpDZOv5SAjWMwmOjcjTChruhQj6o3rK6O5ROA5mlA9MLx0Zth4S8ljG"
genius = lg.Genius(CLIENT_ACCESS_TOKEN)


Degrees of similarity for lyrics data:
- annotation text and the respective lyrics text portion should have high similarity

In [15]:
def lyrics_annotations_pairs_gen(genius_song_id):

    annotations_sample_song = genius.song_annotations(genius_song_id)

    l_pairs = []

    for annotation_pair in annotations_sample_song:

        lyrics_ref = annotation_pair[0]

        cond_empty_ref = len(lyrics_ref) <1 or lyrics_ref.isspace()
        cond_ref_chorus = re.match(r"(?i)^\[chorus\s*\d*\]$", lyrics_ref)
        cond_ref_verse = re.match(r"(?i)^\[verse\s*\d*\]$", lyrics_ref)
        
        # Ignore hanlding for "empty reference"
        if cond_empty_ref or cond_ref_chorus or cond_ref_verse:

            continue

        annotation = annotation_pair[1][0][0]
        t_pair = (lyrics_ref, annotation)
        l_pairs.append(t_pair)

    return l_pairs

In [53]:
lyrics_annotations_pairs_gen(7260084)

[('There ain’t no gold in this river\n That I’ve been washin’ my hands in forever\n I know there is hope in these waters\n But I can’t bring myself to swim\n When I am drowning in this silence\n Baby, let me in',
  'In the opening verse of “Easy On Me,” Adele recalls how her relationship with Simon Konecki had reached a breaking point. After six years of dating, Adele indirectly announced her marriage to Simon Konecki at the  59th Annual Grammy Awards in 2017.  However, in 2019, her team formally shared the news of their separation to Associated Press.\n\nAdele states in the opening lyric that she no longer finds value in the “river” which is a metaphor for her relationship. Based on her interview with Vogue USA where she shared that the album is about “divorcing myself,” the “river” can also be a reference to her life. Despite constantly trying to make her married life work and remain happy in doing so, she no longer found value in her life in that state.\n\nIn her interview with Vogu

In [16]:
# Get song description text
sample_song = genius.song(7260084)
sample_song['song']['description']['plain']

'“Easy On Me” marks Adele’s return after a six-year hiatus. The song is the lead single off her fourth studio album 30. In an October 2021 BBC Radio 1 interview, Adele revealed that it was the first song written for the album, as she started it back in 2019. The song sees the singer speak about her fraught childhood, her lost marriage and the lessons learnt and unlearnt about family, love and abandonment along the way.\n\nFollowing recent teases of her long-awaited fourth album on October 5, 2021, Adele took to Twitter, announcing that she would be dropping a brand new song on October 15th, sharing the teaser trailer to this track. The first verse of the song was played during Adele’s Instagram Live on October 9, 2021.\n\nUpon its release, “Easy On Me” broke the world record for the most streamed song in 24 hours, with 24 million streams. This unseated the previously record-holders BTS for their song “Butter.”'

In [5]:
df = pd.read_csv('../raw_data_local/songsdata_combined.csv')
s_songs_id = pd.to_numeric(df['genius_id'], errors='coerce')
list_songs_id = s_songs_id.astype('Int64').to_list()

In [6]:
from finetune_pairs import lyrics_annotations_gen_id
l_pairs, l_genius_song_idx = lyrics_annotations_gen_id(list_songs_id)

In [8]:
len(l_pairs)

38782

In [9]:
len(l_genius_song_idx)

38782

In [13]:
l_pickle = [l_pairs, l_genius_song_idx]
with open('../pickle_objects/lyrics_annotation_pairs.obj', 'wb') as f:
    pickle.dump(l_pickle, f)

Other dimensions to be considered

- lyrics text with same artist/ same song-writer to have high similarity
- lyrics text in the same song to have high similarity
- lyrics text in the same verse should have high similarity